<a href="https://colab.research.google.com/github/Neafiol/Tinkoff/blob/master/theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Контрольные вопросы

1. Приведите 2 примера слоёв (и как конкретно они работают), поведение которых отличается во время обучения и инференса.

**Ответ:**

- dropout, во время обучения его коэффициенты равны 0|1, во время предсказания всегда 1
- batch normal, во время обучения он перевычесляет _median_ и _b_ на каждом проходе, во время предсказания использует сохраненные константы

2. Зачем нужен батч? Почему бы не использовать весь датасет? Почему бы не использовать один пример? Почему бы не прогнать сеть на $n$ примерах независимо, а затем усреднить градиенты параметров?

**Ответ:**
- Бач нужен что бы ускорить обучение и сделать изменение градиента более плавным.
- Если просчитать loss для всего датасета и потом сделать шаг оптимизатора то мы просто сделаем 1 максимально усредненный шаг в каком-то направлении и можем не изучить особенности связаные с конкреиными случаями в данных, так же обучение на всем датасете требует очень больших ресурсов. 
- Прогнав модель на одном примере мы вообще ничего не сможем обучить так  как никаких зависимостей данных - результатов нет, даже разгых результатов нет, модель просто научится выводить константу.
- Высчитывать градиент после каждого прогона слишком затратно

3. Что не так с этим примером кода?

```python
history = []
for X, Y in dataloader:
    preds = model(X)
    
    loss = criterion(preds, Y)
    loss.backward()
    
    history.append(loss)
```

**Ответ:**

Смотря что хотел автор), если обучить модель, то он забыл `model.backward()` и `model.zero_grad()`

4. Почему не имеет смысл использовать два линейных слоя подряд? 

**Ответ:**
- При перемножение весов  2ух слоев каждый нейрон все все равно прсто приобрет какой - то вес и смещение, которого можно добиться используя только один слой, следовательно 2 Liner подряд - просто пустая трата ресурсов компьютера

5. Почему люди отказались от сигмоиды как от функции активации? Какие у неё недостатки?

**Ответ:**

Она нечувствительна к ненормированым данным, все большые числа -> 1, маленикие к 0 в результате она может плохо различать разные случаи с большим значениями на выходе.

6. Параметры сети весят 100MB. Сколько дополнительной памяти потребуется на хранение параметров, если при обучении использовать Adam?

**Ответ:**

- 196.34 Мб

7. Чем латентные переменные отличаются от параметров модели? На примере задачи кластеризации (k-means), что является латентными переменными, а что параметрами?

**Ответ:**

- латентные переменные подбираются и изменяются во время обучения, в то время как параметры это костнанты, закрепленные во время инициализации модели

- параметр : `n_clusters` - количество классов, латентные переменные : U$k$ - центры кластеров элементов, принадлежащих одному классу

8. Как использовать VAE для генерации цифр MNIST конкретного класса? Разрешается обучить только одну сеть.

**Ответ:**

- добавить между `encoder` и `decoder`2 паралельных и 1 последовательный `Linear()` в которых реализовать оценку близости нейронов, кодирующих одинаковые цифры на скрытом слое.
                                            ↓
`torch.randn(nn.Linear(h_dim, z_dim).size())`   *   `nn.Linear(h_dim, z_dim).mul(0.5).exp_()` + `nn.Linear(h_dim, z_dim)`
                                            ↓
                             self.z2h=nn.Linear(z_dim,h_dim)
                                            ↓
                                           ...

9. Как сравнить качество двух разных GAN-ов, обученых на ImageNet?

**Ответ:**

- Глазами
- Обучить модель, которая должна определять реальная перед ней картика или сгенерированная

10. (Сложно) Пусть есть сеть из $n$ линейных слоев одинакового размера с сигмоидой в качестве функции активации после каждого слоя. Обычно, мы сохраняем выход каждого слоя, из-за чего нам требуется $O(n)$ дополнительной памяти на каждый пример в батче. Можно ли обучить такую сеть, использая $O(1)$ дополнительной памяти, оставив асимптотику по времени $O(n)$?

**Ответ:**

- можно не сохранять, а передовать результат проеобразования каждого слоя по ссылке, таким образом на нужно будет хранить только 1 матрицу и т.к. размер всех слоев одинаковый ее размер не будет меняться во время прохода по nn